# We vary the capacity of the Bayesian model by changing `hidden_size` parameter, trying to predict the brownian trajectories with a lot of noise.

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import torch

In [ ]:
from brownian_datagen import BrownianDatagen, create_dataholder_for_windows_and_targets
from bayesian import BayesTrainData
from bayesian_varinf import train_fb_n_times, load_learning_curves

In [ ]:
def get_save_dir_prefix(hidden_size: int) -> str:
    save_dir_prefix = "20230724_preds/brownian_varinf_hidden_size_noisy/"
    return os.path.join(save_dir_prefix, f"hidden_size={hidden_size:02}/")

In [ ]:
brownian = BrownianDatagen(kBT=1., γ=1., k=1., λ_τ=5., τ=10.)
dt = create_dataholder_for_windows_and_targets(brownian, numParticles=50, rng_seed=42)
brownian.visualize()

In [ ]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    print(f"Number of available GPUs: {num_gpus}")
    for i in range(num_gpus):
        print(f"GPU {i}")
    assert torch.cuda.is_available()
else:
    print("No GPUs available. Using CPU.")
    assert not torch.cuda.is_available()

In [ ]:
for hidden_size in (2, 4, 6, 8, 11, 16):
    save_dir_prefix = get_save_dir_prefix(hidden_size)
    if not os.path.isdir(save_dir_prefix):
        train_fb_n_times(dt,
            num_runs=100,
            hidden_size=hidden_size,
            num_epochs=4000,
            save_metrics_every_n_epochs=100,
            train_test_split_ratio=None,
            save_dir_prefix=save_dir_prefix,
            torch_device="cuda:1" if torch.cuda.is_available() else None)
    else:
        print(f"Directory '{save_dir_prefix}' exists, assuming the previous results are reusable")